In [0]:

df = spark.read.csv(
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/employee.csv",
    header=True,inferSchema=True,sep='|',quote="'"
).limit(10)

# Array Type

## Create an Array type Column

In [0]:
from pyspark.sql.functions import split
result_df=df.withColumn("skills",split("col_skills",","))
result_df.printSchema()

In [0]:
from pyspark.sql.functions import split
 
result_df = (
    df.withColumn("skills", split("col_skills", ","))
    .withColumn(
        "current_expected_salary",
        split("col_current_expected_salary", ",").cast("array<int>"),
    )
    .drop("col_skills", "col_current_expected_salary")
)
result_df.printSchema()
result_df.display()

# Accessing Array

In [0]:
from pyspark.sql.functions import col
   
result_df.select(col("skills"), col("skills").getItem(0),col("skills")[0]).display()

# Derive 2 Columns
First element -- > current_salary
second element -->  expected_slary

In [0]:
from pyspark.sql.functions import col 
result_df.select(
    col("current_expected_salary"),
    col("current_expected_salary")[0].alias("current_salary"),
    col("current_expected_salary")[1].alias("expected_salary"),
).display()

## Fetch Employee Name whose current_salary  > Expected Salary

In [0]:
from pyspark.sql.functions import col

df2 = result_df.select(
    col("name"),
    col("current_expected_salary")[0].alias("current_salary"),
    col("current_expected_salary")[1].alias("expected_salary")
)

df2.filter(col("current_salary") > col("expected_salary")).display()

In [0]:
from pyspark.sql.functions import size, array_distinct, array_contains
 
result_df.select(
    col("name"),
    col("skills"),
    size("skills"),
    array_contains("skills", "PySpark"),
    array_distinct("skills").alias("distinct_skills"),
).display()
 

In [0]:
from pyspark.sql.functions import size, array_distinct, array_contains
 
result_df.select(
    col("name"),     
    array_contains("skills", "PySpark").alias("contains_PySpark"),   
).filter(col("contains_PySpark")==True).display()

In [0]:
from pyspark.sql.functions import size, array_distinct, array_contains
 
result_df.select(
    col("name"),     
    array_contains("skills", "PySpark").alias("contains_PySpark"),   
).filter(col("contains_PySpark")==True).display()


In [0]:

from pyspark.sql.functions import when
result_df.select(
    col("name"),     
    array_contains("skills", "PySpark").alias("contains_PySpark"),   
    col("current_expected_salary")[1].alias("expected_salary")
).withColumn("offeredSalary",when(col("contains_PySpark")==True,col("expected_salary")+col("expected_salary")*0.3)           
              .otherwise(col("expected_salary")),
                              ).display()